In [106]:
## IMPORTACIÓN GENERAL DE LIBRERIAS.
import re 
import sys
import json
import nltk
import geocoder
import requests
import warnings
import descartes

import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import datetime as DT
import category_encoders as ce
import matplotlib.pyplot as plt

# SKLEARN.
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# RANDOM FOREST.
from urllib.request import urlopen
from shapely.geometry import Point, Polygon
from sklearn.ensemble import RandomForestRegressor

# XGBOOST.
from xgboost import XGBClassifier
import xgboost as xgb

In [107]:
#NLTK: https://www.nltk.org
from langdetect import detect
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

#WORDCLOUD
from PIL import Image
from wordcloud import WordCloud

# CONFIGURACIÓN.
%matplotlib inline
plt.style.use('default')
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
np.set_printoptions(threshold=sys.maxsize)

In [108]:
import re
import json
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from textblob import TextBlob

In [112]:
#Limpia puntuacion, quita también usuarios y hashtags (@ y #)
def clean_tweet(words):
    result = []
    for word in words:
        stripped_word = word.strip()
        if ((stripped_word.isalnum() == True) and (not(stripped_word.isdigit()))):      
            result.append(stripped_word)
    
    return result

def filtrarPalabras(miArray):
    variable = ''
    for key in miArray:
         if '#' in str(key):
            variable = variable + ' ' + str(key)
    return variable

### =====================================================================
### LECTURA DE CSV.
### =====================================================================

In [113]:
#Importacion del archivo CSV de fuente
#https://www.kaggle.com/c/nlp-getting-started
original_train = pd.read_csv('data/train.csv')
original_test = pd.read_csv('data/test.csv')
original_sample_submission = pd.read_csv('data/sample_submission.csv')

### =====================================================================
### PROCESAMIENTO DE DATOS.
### =====================================================================

In [114]:
#COMPLETAMOS LOS NULOS CON UN TEXTO: 'VACIO'
original_train.fillna('vacio', inplace = True)
original_test.fillna('vacio', inplace = True)

In [115]:
original_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7613 non-null   object
 2   location  7613 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [116]:
#PASAMOS TODO A MINÚSCULAS (TRAIN).
original_train['text'] = original_train['text'].str.lower()
original_train['location'] = original_train['location'].str.lower()
original_train['keyword'] = original_train['keyword'].str.lower()
#PASAMOS TODO A MINÚSCULAS (TEST).
original_test['text'] = original_test['text'].str.lower()
original_test['location'] = original_test['location'].str.lower()
original_test['keyword'] = original_test['keyword'].str.lower()

In [117]:
## ==================================================
## TRATAMIENTO INCLUYENDO STOP WORDS.
## ==================================================

In [118]:
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TRAIN).
original_train['length'] = original_train['text'].str.len()
original_train['totalwords'] = original_train['text'].str.split().str.len()
original_train['words'] = original_train.text.str.strip().str.split()
original_train['lenXword'] = (original_train['length'] / original_train['totalwords'])
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TEST).
original_test['length'] = original_test['text'].str.len()
original_test['totalwords'] = original_test['text'].str.split().str.len()
original_test['words'] = original_test.text.str.strip().str.split()
original_test['lenXword'] = (original_test['length'] / original_test['totalwords'])

In [119]:
#ARMANDO UNA LISTA DE PALABRAS QUE PUEDEN REPRESENTAR CATÁSTROFES (TRAIN & TEST).
selected_words_singular=['fire','flood','inundate','earthquake','quake','deluge','euption','twister','tornado','hurricane', 'landslide','typhoon','wildfire','forest fire','drought','avalanche','urgent','important','danger','warrning','evacuation']
selected_words_plural=['fires','floods', 'earthquakes','quakes','deluges','rashes','tornadoes','hurricanes', 'landslides','typhoons','wildfires','forest fires','droughts','avalanches']
selected_words_other=['heat wave','died','flooding','flooded','damage','urgent','important','danger','warrning','help','evacuation']
col_one_list = original_train['keyword'].tolist()
selected_words = selected_words_singular + selected_words_plural + selected_words_other + col_one_list
s = set(selected_words)

In [120]:
#BUSCAMOS LAS COINCIDENCIAS CON PALABRAS CLAVES Y SEPARAMOS LOS HASHTAGS (TRAIN).
original_train = original_train.assign(hashtags=[filtrarPalabras(el) for el in original_train.words])
original_train = original_train.assign(matches=[len(set(el) & s) for el in original_train.words])
#BUSCAMOS LAS COINCIDENCIAS CON PALABRAS CLAVES Y SEPARAMOS LOS HASHTAGS (TEST).
original_test = original_test.assign(hashtags=[filtrarPalabras(el) for el in original_test.words])
original_test = original_test.assign(matches=[len(set(el) & s) for el in original_test.words])

In [121]:
#CONTAMOS LA CANTIDAD DE HASHTAGS (TRAIN).
original_train['hashtagsCantidad'] = original_train['hashtags'].str.count('#')
original_train['preguntas'] = original_train['text'].str.count('[!]') + original_train['text'].str.count('[?]')
original_train['simbolos'] = original_train['text'].str.count('[!]') + original_train['text'].str.count('[?]') + original_train['text'].str.count('=') + original_train['text'].str.count('>')
#CONTAMOS LA CANTIDAD DE HASHTAGS (TEST).
original_test['hashtagsCantidad'] = original_test['hashtags'].str.count('#')
original_test['preguntas'] = original_test['text'].str.count('[!]') + original_test['text'].str.count('[?]')
original_test['simbolos'] = original_test['text'].str.count('[!]') + original_test['text'].str.count('[?]') + original_test['text'].str.count('=') + original_test['text'].str.count('>')

In [122]:
#MARCAMOS QUIENES USAN UBICACIÓN Y PALABRAS CLAVES (TRAIN).
original_train['conUbicacion'] = 0
original_train['conKeyword'] = 0
original_train.loc[original_train['location'] != 'vacio', ['conUbicacion']] = 1
original_train.loc[original_train['keyword'] != 'vacio', ['conKeyword']] = 1
#MARCAMOS QUIENES USAN UBICACIÓN Y PALABRAS CLAVES (TEST).
original_test['conUbicacion'] = 0
original_test['conKeyword'] = 0
original_test.loc[original_test['location'] != 'vacio', ['conUbicacion']] = 1
original_test.loc[original_test['keyword'] != 'vacio', ['conKeyword']] = 1

In [123]:
## ==================================================
## TRATAMIENTO QUITANDO LAS STOP WORDS.
## ==================================================

In [124]:
# Tratamiento del set con nltk y nlpaug
# NLTK
# Sabiendo que todos los tweets son en idioma ingles, quitamos las stopwords
stop_words = set(stopwords.words('english'))
original_train['text'] =  original_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
original_test['text'] =  original_test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [125]:
#PASAMOS A LA RAÍZ IDOMÁTICA DE LA PALABRA.
stemmer = SnowballStemmer('english')
original_train['text'] = original_train['text'].apply(lambda x:' '.join([stemmer.stem(y) for y in x.split()]))
original_test['text'] = original_test['text'].apply(lambda x:' '.join([stemmer.stem(y) for y in x.split()]))

In [126]:
#QUITAMOS LOS NÚMEROS.
original_train['text'] = original_train['text'].str.replace('\d+', '')
original_test['text'] = original_test['text'].str.replace('\d+', '')
original_train['text'] = original_train['text'].str.replace('vacio', '')
original_test['text'] = original_test['text'].str.replace('vacio', '')

In [127]:
#TF-IDF
original_train_Serie = original_train['text']
original_train_DF = original_train_Serie.to_frame()

In [128]:
original_train_DF.head()

,text
0,deed reason #earthquak may allah forgiv us
1,forest fire near la rong sask. canada
2,resid ask shelter place notifi officers. evacu...
3,", peopl receiv #wildfir evacu order california"
4,got sent photo rubi #alaska smoke #wildfir pou...


In [129]:
#TF-IDF
original_train_Serie = original_train['text']
original_train_DF = original_train_Serie.to_frame()

tfidf = TfidfVectorizer(min_df=2,max_df=0.5, ngram_range=(1,3))
features = tfidf.fit_transform(original_train_DF['text'])
original_train_DF = pd.concat(
    [
        original_train_DF,
        pd.DataFrame(
            features.todense(),
            columns=tfidf.get_feature_names()
        )
    ], axis=1
)

In [130]:
#TF-IDF
original_test_Serie = original_test['text']
original_test_DF = original_test_Serie.to_frame()

tfidf = TfidfVectorizer(min_df=2,max_df=0.75, ngram_range=(1,3))
features = tfidf.fit_transform(original_test_DF['text'])
original_test_DF = pd.concat(
    [
        original_test_DF,
        pd.DataFrame(
            features.todense(),
            columns=tfidf.get_feature_names()
        )
    ], axis=1
)

In [100]:
original_train_DF = original_train.fillna(0)
original_test_DF = original_test.fillna(0)

In [132]:
original_train_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Columns: 16230 entries, text to ûó stori volga
dtypes: float64(16229), object(1)
memory usage: 942.7+ MB


In [136]:
def get_correlation(data, threshold):
    corr_col = set()
    cormat = data.corr()
    for i in range(len(cormat.columns)):
        for j in range(i):
            if abs(cormat.iloc[i,j]) > threshold:
                colname = cormat.columns[i]
                corr_col.add(colname)
    return corr_col

corr_features = get_correlation(original_train_DF, 0.70)

KeyboardInterrupt: 

In [ ]:
original_train_DF = original_train_DF.drop(labels= corr_features, axis = 1)

In [15]:
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TRAIN).
original_train['length_SW'] = original_train['text'].str.len()
original_train['totalwords_SW'] = original_train['text'].str.split().str.len()
original_train['lenXword_SW'] = (original_train['length_SW'] / original_train['totalwords_SW'])
original_train['stopWords'] = (original_train['totalwords'] - original_train['totalwords_SW']) + 1
original_train['totalXsw'] = (original_train['totalwords_SW'] / original_train['stopWords'])
#LONGITUD DEL TWEET Y CANTIDAD DE PALABRAS (TEST).
original_test['length_SW'] = original_test['text'].str.len()
original_test['totalwords_SW'] = original_test['text'].str.split().str.len()
original_test['lenXword_SW'] = (original_test['length_SW'] / original_test['totalwords_SW'])
original_test['stopWords'] = (original_test['totalwords'] - original_test['totalwords_SW']) + 1
original_test['totalXsw'] = (original_test['totalwords_SW'] / original_test['stopWords'])

In [16]:
## ==================================================
## ENCODING
## ==================================================

In [17]:
encoder = ce.BinaryEncoder(cols=['keyword'])
original_train_ce = encoder.fit_transform(original_train)
original_train_ce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                7613 non-null   int64  
 1   keyword_0         7613 non-null   int64  
 2   keyword_1         7613 non-null   int64  
 3   keyword_2         7613 non-null   int64  
 4   keyword_3         7613 non-null   int64  
 5   keyword_4         7613 non-null   int64  
 6   keyword_5         7613 non-null   int64  
 7   keyword_6         7613 non-null   int64  
 8   keyword_7         7613 non-null   int64  
 9   keyword_8         7613 non-null   int64  
 10  location          7613 non-null   object 
 11  text              7613 non-null   object 
 12  target            7613 non-null   int64  
 13  length            7613 non-null   int64  
 14  totalwords        7613 non-null   int64  
 15  words             7613 non-null   object 
 16  lenXword          7613 non-null   float64


In [18]:
encoder = ce.BinaryEncoder(cols=['keyword'])
original_test_ce = encoder.fit_transform(original_test)
original_test_ce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3263 non-null   int64  
 1   keyword_0         3263 non-null   int64  
 2   keyword_1         3263 non-null   int64  
 3   keyword_2         3263 non-null   int64  
 4   keyword_3         3263 non-null   int64  
 5   keyword_4         3263 non-null   int64  
 6   keyword_5         3263 non-null   int64  
 7   keyword_6         3263 non-null   int64  
 8   keyword_7         3263 non-null   int64  
 9   keyword_8         3263 non-null   int64  
 10  location          3263 non-null   object 
 11  text              3263 non-null   object 
 12  length            3263 non-null   int64  
 13  totalwords        3263 non-null   int64  
 14  words             3263 non-null   object 
 15  lenXword          3263 non-null   float64
 16  hashtags          3263 non-null   object 


In [19]:
#HOT ENCODING PARA KEYWORD (TRAIN).
# dummies = pd.get_dummies(original_train['keyword'], drop_first=False)
# original_train = pd.concat([original_train, dummies], axis=1)
# original_train.drop('keyword', 1, inplace = True)

#HOT ENCODING PARA KEYWORD (TEST).
# dummies = pd.get_dummies(original_test['keyword'], drop_first=False)
# original_test = pd.concat([original_test, dummies], axis=1)
# original_test.drop('keyword', 1, inplace = True)

In [20]:
## ==================================================
## DEJAMOS SOLO LAS COLUMNAS NUMÉRICAS.
## ==================================================

In [21]:
#ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS (TRAIN).
original_train_ce.drop('text', 1, inplace = True)
original_train_ce.drop('words', 1, inplace = True)
original_train_ce.drop('location', 1, inplace = True)
original_train_ce.drop('hashtags', 1, inplace = True)
#ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS (TEST).
original_test_ce.drop('text', 1, inplace = True)
original_test_ce.drop('words', 1, inplace = True)
original_test_ce.drop('location', 1, inplace = True)
original_test_ce.drop('hashtags', 1, inplace = True)

### =====================================================================
### RESGUARDAMOS LOS DATOS EN CSV.
### =====================================================================

In [22]:
#Vemos la estructura del dataframe TRAIN.
original_train_ce.head()

,id,keyword_0,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,...,hashtagsCantidad,preguntas,simbolos,conUbicacion,conKeyword,length_SW,totalwords_SW,lenXword_SW,stopWords,totalXsw
0,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,45,7,6.43,7,1.00
1,4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,38,7,5.43,1,7.00
2,5,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,91,11,8.27,12,0.92
3,6,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,61,7,8.71,2,3.50
4,7,0,0,0,0,0,0,0,0,1,...,2,0,0,0,0,57,9,6.33,8,1.12


In [23]:
original_train_ce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                7613 non-null   int64  
 1   keyword_0         7613 non-null   int64  
 2   keyword_1         7613 non-null   int64  
 3   keyword_2         7613 non-null   int64  
 4   keyword_3         7613 non-null   int64  
 5   keyword_4         7613 non-null   int64  
 6   keyword_5         7613 non-null   int64  
 7   keyword_6         7613 non-null   int64  
 8   keyword_7         7613 non-null   int64  
 9   keyword_8         7613 non-null   int64  
 10  target            7613 non-null   int64  
 11  length            7613 non-null   int64  
 12  totalwords        7613 non-null   int64  
 13  lenXword          7613 non-null   float64
 14  matches           7613 non-null   int64  
 15  hashtagsCantidad  7613 non-null   int64  
 16  preguntas         7613 non-null   int64  


In [24]:
#Vemos la estructura del dataframe TEST.
original_test_ce.head()

,id,keyword_0,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,...,hashtagsCantidad,preguntas,simbolos,conUbicacion,conKeyword,length_SW,totalwords_SW,lenXword_SW,stopWords,totalXsw
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,27,4,6.75,3,1.33
1,2,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,55,7,7.86,3,2.33
2,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,63,10,6.30,10,1.00
3,9,0,0,0,0,0,0,0,0,1,...,2,0,0,0,0,40,4,10.00,1,4.00
4,11,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,38,6,6.33,3,2.00


In [25]:
original_test_ce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3263 non-null   int64  
 1   keyword_0         3263 non-null   int64  
 2   keyword_1         3263 non-null   int64  
 3   keyword_2         3263 non-null   int64  
 4   keyword_3         3263 non-null   int64  
 5   keyword_4         3263 non-null   int64  
 6   keyword_5         3263 non-null   int64  
 7   keyword_6         3263 non-null   int64  
 8   keyword_7         3263 non-null   int64  
 9   keyword_8         3263 non-null   int64  
 10  length            3263 non-null   int64  
 11  totalwords        3263 non-null   int64  
 12  lenXword          3263 non-null   float64
 13  matches           3263 non-null   int64  
 14  hashtagsCantidad  3263 non-null   int64  
 15  preguntas         3263 non-null   int64  
 16  simbolos          3263 non-null   int64  


In [26]:
#Vemos la estructura del dataframe SAMPLE_SUBMISSION.
original_sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [27]:
original_train_ce.to_csv('data/processed/original_train.csv',index=False)
original_test_ce.to_csv('data/processed/original_test.csv',index=False)

### =====================================================================
### FIN.
### =====================================================================